In [ ]:
from heapq import heappush, heappop
from dataclasses import dataclass
from typing import List, Set, Tuple, Dict
from array import array

In [ ]:
@dataclass
class Node:
    x: int
    y: int
    g: int = 0
    parent: 'Node' = None

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        return self.x * 1000 + self.y  # Faster than tuple hash

In [ ]:
class PathFinder:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        # Use array for obstacle grid - much faster than set
        self.grid = [[False] * height for _ in range(width)]
        # Pre-allocate neighbor offsets
        self.directions = [(0,1), (1,0), (0,-1), (-1,0)]

    def load_obstacles(self, filename: str, limit: int):
        with open(filename) as f:
            for n, line in enumerate(f):
                if n >= limit:
                    break
                x, y = map(int, line.strip().split(','))
                self.grid[x][y] = True

    def manhattan_distance(self, x1: int, y1: int, x2: int, y2: int) -> int:
        return abs(x1 - x2) + abs(y1 - y2)

    def find_path(
            self,
            start: Tuple[int, int],
            goal: Tuple[int, int]
        ) -> List[Tuple[int, int]]:

        start_x, start_y = start
        goal_x, goal_y = goal

        # Use dictionary to track g scores
        g_scores: Dict[Tuple[int, int], int] = {(start_x, start_y): 0}

        # Use tuples in heap: (f_score, g_score, x, y, parent_x, parent_y)
        open_set = [(0, 0, start_x, start_y, -1, -1)]

        # Track parents for reconstruction
        came_from: Dict[Tuple[int, int], Tuple[int, int]] = {}

        while open_set:
            f, g, x, y, parent_x, parent_y = heappop(open_set)

            current_pos = (x, y)

            # Found goal
            if x == goal_x and y == goal_y:
                path = []
                while current_pos in came_from:
                    path.append(current_pos)
                    current_pos = came_from[current_pos]
                path.append((start_x, start_y))
                return path[::-1]

            # Check neighbors
            for dx, dy in self.directions:
                new_x, new_y = x + dx, y + dy

                if not (0 <= new_x < self.width and 0 <= new_y < self.height):
                    continue

                if self.grid[new_x][new_y]:
                    continue

                neighbor_pos = (new_x, new_y)
                tentative_g = g + 1

                if neighbor_pos in g_scores and tentative_g >= g_scores[neighbor_pos]:
                    continue

                came_from[neighbor_pos] = current_pos
                g_scores[neighbor_pos] = tentative_g
                f = tentative_g + self.manhattan_distance(new_x, new_y, goal_x, goal_y)

                heappush(open_set, (f, tentative_g, new_x, new_y, x, y))

        return []

In [ ]:
# file = "example"
# width = 7
# height = 7
# limit = 12

file = "input"
width = 71
height = 71
limit = 1024

In [ ]:
finder = PathFinder(width, height)
finder.load_obstacles(file, limit)
path = finder.find_path((0,0), (width-1, height-1))

In [ ]:
path

In [ ]:
len(path) - 1

In [ ]:
def find_breaking_obstacle(
        filename: str,
        width: int,
        height: int,
    ) -> Tuple[int, int]:

    with open(filename) as f:
        lines = f.readlines()
    max_limit = len(lines)

    left = 0
    right = max_limit

    while left < right:
        mid = (left + right + 1) // 2

        finder = PathFinder(width, height)
        finder.load_obstacles(filename, mid)
        path = finder.find_path((0,0), (width-1, height-1))

        if path:  # Path exists
            left = mid
        else:  # No path
            right = mid - 1

    # left is now the last working limit
    # breaking obstacle is at index left + 1
    for i, line in enumerate(lines):
        if i == left:
            x, y = map(int, line.strip().split(','))
            return x, y

    return -1, -1  # Should not reach here

In [ ]:
find_breaking_obstacle(file, width, height)